In [1]:
# !pip install h3
import duckdb
import h3

In [2]:
#add unsigned flag as using some extensions that are not approved yet aka H3
db = duckdb.connect("data.db --unsigned")

#add spatial extension to work with spatial data formats like SHP and GEOJSON
db.sql("""
    INSTALL spatial;
    LOAD spatial;
""")

In [3]:
#reading in csv with standard duckdb
duckdb.read_csv('data-duck/trees-with-species-and-dimensions-urban-forest.csv')

┌─────────┬──────────────────────┬──────────────────────┬───┬───────────┬────────────┬──────────────────────┐
│ com_id  │     common_name      │   scientific_name    │ … │  easting  │  northing  │     geolocation      │
│  int64  │       varchar        │       varchar        │   │  double   │   double   │       varchar        │
├─────────┼──────────────────────┼──────────────────────┼───┼───────────┼────────────┼──────────────────────┤
│ 1440992 │ River red gum        │ Eucalyptus camaldu…  │ … │ 319271.37 │ 5815606.69 │ -37.789042536009, …  │
│ 1286119 │ River red gum        │ Eucalyptus camaldu…  │ … │ 317353.24 │ 5815470.25 │ -37.78989006812276…  │
│ 1439848 │ European nettle tree │ Celtis australis     │ … │ 316812.46 │ 5814865.65 │ -37.79522759209887…  │
│ 1584631 │ Swamp Sheoak         │ Casuarina obesa      │ … │ 319713.76 │ 5814935.15 │ -37.79517879825104…  │
│ 1286271 │ Golden Poplar        │ Populus x canadensis │ … │ 317538.99 │ 5815415.81 │ -37.7904175404039,…  │
│ 1288100 

In [4]:
#read in the geojson version of the Urban Forest trees data. This is the same info as in the csv
#was getting error 'table already exists' when re-running cell so added 'create or replace'
db.sql("""
create or replace table trees as
       select * from ST_read("data-duck/trees-with-species-and-dimensions-urban-forest.geojson")
""")

db.sql("""
create or replace table precincts as
       select * from ST_read("data-duck/tree-planting-zone-schedules-with-years-urban-forest.geojson")
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
db.sql("""
select * from trees limit 5
""")

┌─────────┬──────────────────────┬──────────────────────┬───┬────────────┬──────────────────────┬──────────────────────┐
│ com_id  │     common_name      │   scientific_name    │ … │  northing  │     geolocation      │         geom         │
│  int32  │       varchar        │       varchar        │   │   double   │       varchar        │       geometry       │
├─────────┼──────────────────────┼──────────────────────┼───┼────────────┼──────────────────────┼──────────────────────┤
│ 1440992 │ River red gum        │ Eucalyptus camaldu…  │ … │ 5815606.69 │ { "lon": 144.94750…  │ POINT (144.9475011…  │
│ 1286119 │ River red gum        │ Eucalyptus camaldu…  │ … │ 5815470.25 │ { "lon": 144.92569…  │ POINT (144.9256959…  │
│ 1439848 │ European nettle tree │ Celtis australis     │ … │ 5814865.65 │ { "lon": 144.91940…  │ POINT (144.9194053…  │
│ 1584631 │ Swamp Sheoak         │ Casuarina obesa      │ … │ 5814935.15 │ { "lon": 144.95235…  │ POINT (144.9523553…  │
│ 1286271 │ Golden Poplar       

In [6]:
db.sql("""
select * from precincts limit 5
""")

┌──────────────────────┬──────────┬─────────┬──────────┬───┬─────────┬──────────────────────┬──────────────────────┐
│     geo_point_2d     │ str_from │ segpart │ statusid │ … │ str_to  │       segdescr       │         geom         │
│       varchar        │ varchar  │ varchar │ varchar  │   │ varchar │       varchar        │       geometry       │
├──────────────────────┼──────────┼─────────┼──────────┼───┼─────────┼──────────────────────┼──────────────────────┤
│ { "lon": 144.96879…  │ NULL     │ NULL    │ NULL     │ … │ NULL    │ Pelham Street betw…  │ MULTIPOLYGON (((14…  │
│ { "lon": 144.96751…  │ NULL     │ NULL    │ NULL     │ … │ NULL    │ Riverside Avenue b…  │ MULTIPOLYGON (((14…  │
│ { "lon": 144.96990…  │ NULL     │ NULL    │ NULL     │ … │ NULL    │ Little Palmerston …  │ MULTIPOLYGON (((14…  │
│ { "lon": 144.95026…  │ NULL     │ NULL    │ NULL     │ … │ NULL    │ Chapman Street bet…  │ MULTIPOLYGON (((14…  │
│ { "lon": 144.98649…  │ NULL     │ NULL    │ NULL     │ … │ NUL

In [7]:
#use describe to see all the columns as the above select statement doesn't display them all
db.sql("""
describe trees
""")

┌──────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│         column_name          │ column_type │  null   │   key   │ default │ extra │
│           varchar            │   varchar   │ varchar │ varchar │ varchar │ int32 │
├──────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ com_id                       │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ common_name                  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ scientific_name              │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ genus                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ family                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ diameter_breast_height       │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ year_planted                 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ date_planted                 │ DATE        │ YES     │ NULL    

In [8]:
db.sql("""
describe precincts
""")

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name  │ column_type │  null   │   key   │ default │ extra │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ int32 │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ geo_point_2d │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ str_from     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ segpart      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ statusid     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ segid        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ streetname   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ streetid     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ schedule     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ mapstatus    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ str_to       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ segdescr     │ VARCHAR     │ YES